<a id='home'></a>
### purpose

save validation scores in a dataframe by simulation level (cell 8) and permuation level (cell 11) for pooled data only

### notes

some of the performance_dicts do not have ind_all (because of 'early finish' - see notes). This should not affect anything downstream of this notebook because all future analysis focuses on pooled

The code is almost identical for the level_scores of the other non-1-trait directories except how it defines the number of traits and causal envs (these are 1-trait sims where GF used only 1 env in training).

In [1]:
minseed = 225
maxseed = 450

assert maxseed - minseed == 225
assert minseed % 225 == 0
assert maxseed % 225 == 0

In [2]:
from pythonimports import *

import MVP_10_train_lfmm2_offset as mvp10
import MVP_summary_functions as mvp

outerdir = f'/work/lotterhos/MVP-Offsets/1-trait_run_20220919_{minseed}-{maxseed}'
assert op.exists(outerdir)
slimdir = f'{outerdir}/slimdir'
pkldir = makedir(f'{outerdir}/summaries/all_performance_dicts')

t0 = dt.now()  # notebook timer

mvp.latest_commit()
session_info.show()

#########################################################
Today:	May 25, 2023 - 13:00:54
python version: 3.8.5
conda env: mvp_env

Current commit of pythonimports:
commit c721bd0d89c1e150e17a4e0739c26e4296405233  
Author: Brandon Lind <lind.brandon.m@gmail.com>  
Date:   Wed May 10 15:13:03 2023 -0400

Current commit of MVP_offsets:
commit 060db40dff8355d2087deab28bd695fcccee8f49  
Author: Brandon Lind <lind.brandon.m@gmail.com>  
Date:   Thu May 18 12:50:56 2023 -0400
#########################################################



In [3]:
programs = ['gradient_forests']

In [4]:
# get directories where performance calculation dicts are saved
valdirs = []
for program in programs:
    d = op.join(outerdir, f'{program}/validation')
    if op.exists(d):
        valdirs.append(d)
        
cordirs = [op.join(d, 'corrs') for d in valdirs]

assert all([op.exists(d) for d in cordirs])

valdirs

['/work/lotterhos/MVP-Offsets/1-trait_run_20220919_225-450/gradient_forests/validation']

In [5]:
# get performance calculations
performance_dicts = defaultdict(dict)
for v in valdirs:
    cordir = op.join(v, 'corrs')
    pkls = fs(cordir, pattern='performance_dicts', endswith='.pkl')
    
    program = v.replace(f'{outerdir}/', '').replace('/validation', '').replace('gradient_forests', 'GF')
    
    for pkl in pbar(pkls, desc=program):
        seed = op.basename(pkl).split("_")[0]
        performance_dicts[program][seed] = pklload(pkl)

GF: 100%|███████████████| 45/45 [00:00<00:00, 611.17it/s]


In [6]:
# original levels within each program's performance dict (for reference when creating dict iterations)

# # rona
# performance_or_slope, marker_set, env = unwrap_dictionary(performance_dicts['RONA'][seed])

# # gf
# performance_or_slope, ind_or_pooled, marker_set = unwrap_dictionary(performance_dicts['GF'][seed])

# # lfmm
# performance_or_slope, marker_set, ntraits = unwrap_dictionary(performance_dicts['lfmm'][seed])

# # rda
# performance_or_slope, ind_or_pooled, marker_set, ntraits, structcrxn = unwrap_dictionary(performance_dicts['rda'][seed])

In [7]:
# reorder GF and RDA so nesting order is: seed, performance_or_slope, marker_set, *args

# keep only pooled data!!!!!!!!!!!!!!!!!!!!!!!!  (remove ind_or_pooled dict level)

gf = wrap_defaultdict(dict, 4)
for (seed, performance_or_slope, ind_or_pooled, marker_set), corrs in unwrap_dictionary(performance_dicts['GF']):
    if ind_or_pooled == 'pooled':
        gf[seed][performance_or_slope][marker_set] = corrs.copy()

# rda = wrap_defaultdict(dict, 4)
# for (seed, performance_or_slope, ind_or_pooled, marker_set, ntraits, structcrxn), corrs in unwrap_dictionary(performance_dicts['rda']):
#     if ind_or_pooled == 'pooled':
#         rda[seed][performance_or_slope][marker_set][ntraits][structcrxn] = corrs.copy()
    
_ = performance_dicts.pop('GF')
# _ = performance_dicts.pop('rda')

performance_dicts['GF'] = gf
# performance_dicts['rda'] = rda

In [8]:
# new levels within each program's performance dict (for reference when creating dict iterations)
    # no ind data! only pooled!

# # rona
# performance_or_slope, marker_set, env = unwrap_dictionary(performance_dicts['RONA'][seed])

# # gf
# performance_or_slope, marker_set = unwrap_dictionary(performance_dicts['GF'][seed])

# # lfmm
# performance_or_slope, marker_set, ntraits = unwrap_dictionary(performance_dicts['lfmm'][seed])

# # rda
# performance_or_slope, marker_set, ntraits, structcrxn = unwrap_dictionary(performance_dicts['rda'][seed])

# gather results

In [9]:
params = mvp10.read_params_file(slimdir)

params.head(3)

level  reps  \
1231094  highly-polygenic_1-trait__Est-Clines_N-cline-c...     1   
1231095  highly-polygenic_1-trait__Est-Clines_N-cline-N...     1   
1231096  highly-polygenic_1-trait__Est-Clines_N-equal_m...     1   

                             arch  \
1231094  highly-polygenic_1-trait   
1231095  highly-polygenic_1-trait   
1231096  highly-polygenic_1-trait   

                                           demog_name  \
1231094  Est-Clines_N-cline-center-to-edge_m-constant   
1231095          Est-Clines_N-cline-N-to-S_m-constant   
1231096                   Est-Clines_N-equal_m_breaks   

                           demog_level_sub demog_level  MIG_x  MIG_y  xcline  \
1231094  N-cline-center-to-edge_m-constant  Est-Clines   0.49   0.07  linear   
1231095          N-cline-N-to-S_m-constant  Est-Clines   0.49   0.07  linear   
1231096                   N-equal_m_breaks  Est-Clines   0.49   0.07  linear   

         ycline    demog  METAPOP_SIDE_x  METAPOP_SIDE_y  Nequal  isVariableM  \
1231094  linear  Estuary              10              10       4            0   
1231095  linear  Estuary              10              10       2            0   
1231096  linear  Estuary              10              10       0            0   

         MIG_breaks arch_level_sub        arch_level       MU_base  \
1231094           0        1-trait  highly-polygenic  1.000000e-07   
1231095           0        1-trait  highly-polygenic  1.000000e-07   
1231096           1        1-trait  highly-polygenic  1.000000e-07   

         MU_QTL_proportion  SIGMA_QTN_1  SIGMA_QTN_2  SIGMA_K_1  SIGMA_K_2  \
1231094               0.25        0.002        0.002        0.5        0.5   
1231095               0.25        0.002        0.002        0.5        0.5   
1231096               0.25        0.002        0.002        0.5        0.5   

         N_traits  ispleiotropy     seed  
1231094         1             0  1231094  
1231095         1             0  1231095  
1231096         1             0  1231096

In [10]:
def create_level_df(scores, seed, marker_set):
    """Create a dataframe that I can group *scores* by sim level (from `params` dataframe)."""
    # get simulation parameters
    glevel, plevel, _blank_, landscape, popsize, *migration = params.loc[seed, 'level'].split("_")
    migration = '-'.join(migration)  # for m_breaks to m-breaks (otherwise m-constant)
    
    assert _blank_ == ''

    # level of pleiotropy and selection
#     pleio = 'no pleiotropy' if '-no-' in plevel else 'pleiotropy'
    if plevel != '1-trait':
        num, trait_str, *pleio, equality, S_str = plevel.split('-')
        plevel = '2-trait'
        pleio = ' '.join(pleio)
        slevel = f'{equality}-S'
    else:
        pleio = 'no pleiotropy'
        slevel = np.nan

    # fill in the performance scores (slope or tau) for each garden (or source_pop if performance=source)
    df = pd.DataFrame(
        dict(garden=keys(scores),
             score=scores.tolist())
    )

    # set single value for each of the locations
    for column, val in zip(
        ['glevel', 'plevel', 'pleio', 'slevel', 'landscape', 'popsize', 'migration', 'marker_set', 'seed'],
        [glevel, plevel, pleio, slevel, landscape, popsize, migration, marker_set, seed]
    ):
        df[column] = val
    
    # set permutation level
    pleio = pleio.replace('no pleiotropy', 'no-pleiotropy')
    level_args = pd.Series([glevel, plevel, pleio, slevel, landscape, popsize, migration]).map(str).tolist()
    df['simulation_level'] = '_'.join(level_args)

    return df


def rda_dict(marker_set):
    """Map RDA marker set names, or return *marker_set* if *marker_set* in `values(marker_dict)`."""
    marker_dict =  {
        'CAUSAL': 'adaptive',
        'TRUE': 'rda outliers',
        'FALSE': 'all',
        'NEUTRAL': 'neutral'
    }
    
    marker_name = marker_set if marker_set not in marker_dict else marker_dict[marker_set]
    
    return  marker_name


def add_bool_ntraits_column(df):
    """Add column that tells whether a non-causal env was included in offset training.
    
    Notes
    -----
    1-trait sims using only 1 env in offset training will be `no noncausal`
    """
    # default value (for 1-trait sims using only 1 env in offset training)
#     df['noncausal_env'] = 'no noncausal'
    df['noncausal_env'] = '1-trait 1-env'
    
    # if sims had one causal env but both envs were used in training
    df.loc[
        (df['plevel']=='1-trait') & (df['ntraits']=='ntraits-2'),
        'noncausal_env'
#     ] = 'one noncausal'
    ] = '1-trait 2-envs'
    
    # if sims had two causal envs and both were used in training (only option for 2-trait sims)
    df.loc[
        (df['plevel']=='2-trait') & (df['ntraits']=='ntraits-2'),
        'noncausal_env'
#     ] = 'all causal'
    ] = '2-trait 2-envs'
    
    return df

@timer
def get_level_scores():
    """Group validation scores by polygenicity and by pleiotropy level."""
    level_dfs = wrap_defaultdict(dict, 2)
    for program in performance_dicts.keys():
        # put dict into dataframes, and dataframes into lists
        level_lists = defaultdict(list)
        for (seed, performance_type, marker_set, *args), corrs in unwrap_dictionary(performance_dicts[program]):
            df = create_level_df(corrs, seed, marker_set)
            
            # add extra dimensions of permutations for each program
            if program == 'RONA':
                df['env'] = args[0]
                
            elif program == 'lfmm2':
                df['ntraits'] = args[0]
                df = add_bool_ntraits_column(df)
                
            elif program == 'rda':
                df['ntraits'] = args[0]
                df['structcrxn'] = args[1]
                df = add_bool_ntraits_column(df)
                
            elif program == 'GF':
                # similar to add_bool_ntraits_column()
#                 df['noncausal_env'] = df.plevel.map({'1-trait' : 'one noncausal', '2-trait' : 'all causal'})
                df['noncausal_env'] = '1-trait 1-env'  # SPECIFIC FOR 1-TRAIT GF RUNS!!!!
            
            level_lists[performance_type].append(df)

        # combine dataframes
        for performance_type, dfs in level_lists.items():
            df = pd.concat(dfs).set_index('seed', drop=False)
            df.index.name = None
            df['marker_set'] = df['marker_set'].apply(rda_dict)  # update RDA marker_set names
            df['offset_level'] = df['simulation_level'] + '_' + df['marker_set']
            level_dfs[program][performance_type] = df

        print(program, keys(level_dfs[program]), '\n')

    return level_dfs

In [11]:
# convert dicts to dataframes, add columns for simulation parameters
level_scores = get_level_scores()

GF ['garden_performance', 'source_performance', 'garden_slopes', 'source_slopes'] 

Function `get_level_scores` completed after : 0-00:00:01


In [12]:
# show dataframes for funzies, do some assertin'
for program in level_scores:
    df = level_scores[program]['garden_performance'].copy()
    
    print(program)
    display(df.head())
    
    if program == 'RONA':
        print('\tenv', Counter(df['env']))
        
    elif program == 'lfmm2':
        print('\tntraits', Counter(df['ntraits']))
        # make sure that 1-trait sims that used 1 env for offset training are all null
#         assert (df.loc[(df.plevel=='1-trait') & (df.ntraits=='ntraits-1'), 'noncausal_env'] == 'no noncausal').all()
        assert (df.loc[(df.plevel=='1-trait') & (df.ntraits=='ntraits-1'), 'noncausal_env'] == '1-trait 1-env').all()
    
    elif program == 'rda':
        print('\tntraits', Counter(df['ntraits']))
        print('\tstructcrxn', Counter(df['structcrxn']))
        # make sure that 1-trait sims that used 1 env for offset training are all null
#         assert (df.loc[(df.plevel=='1-trait') & (df.ntraits=='ntraits-1'), 'noncausal_env'] == 'no noncausal').all()
        assert (df.loc[(df.plevel=='1-trait') & (df.ntraits=='ntraits-1'), 'noncausal_env'] == '1-trait 1-env').all()

GF


garden     score            glevel   plevel          pleio  slevel  \
1231319       1 -0.953463  highly-polygenic  1-trait  no pleiotropy     NaN   
1231319       2 -0.953463  highly-polygenic  1-trait  no pleiotropy     NaN   
1231319       3 -0.953463  highly-polygenic  1-trait  no pleiotropy     NaN   
1231319       4 -0.953463  highly-polygenic  1-trait  no pleiotropy     NaN   
1231319       5 -0.953463  highly-polygenic  1-trait  no pleiotropy     NaN   

          landscape                 popsize   migration marker_set     seed  \
1231319  Est-Clines  N-cline-center-to-edge  m-constant   adaptive  1231319   
1231319  Est-Clines  N-cline-center-to-edge  m-constant   adaptive  1231319   
1231319  Est-Clines  N-cline-center-to-edge  m-constant   adaptive  1231319   
1231319  Est-Clines  N-cline-center-to-edge  m-constant   adaptive  1231319   
1231319  Est-Clines  N-cline-center-to-edge  m-constant   adaptive  1231319   

                                          simulation_level  noncausal_env  \
1231319  highly-polygenic_1-trait_no-pleiotropy_nan_Est...  1-trait 1-env   
1231319  highly-polygenic_1-trait_no-pleiotropy_nan_Est...  1-trait 1-env   
1231319  highly-polygenic_1-trait_no-pleiotropy_nan_Est...  1-trait 1-env   
1231319  highly-polygenic_1-trait_no-pleiotropy_nan_Est...  1-trait 1-env   
1231319  highly-polygenic_1-trait_no-pleiotropy_nan_Est...  1-trait 1-env   

                                              offset_level  
1231319  highly-polygenic_1-trait_no-pleiotropy_nan_Est...  
1231319  highly-polygenic_1-trait_no-pleiotropy_nan_Est...  
1231319  highly-polygenic_1-trait_no-pleiotropy_nan_Est...  
1231319  highly-polygenic_1-trait_no-pleiotropy_nan_Est...  
1231319  highly-polygenic_1-trait_no-pleiotropy_nan_Est...

In [13]:
# make sure levels match seeds
for program in level_scores:
    df = level_scores[program]['garden_performance'].copy()
    print(program, luni(df.seed), luni(df.simulation_level), luni(df.offset_level))

GF 45 45 135


# save

In [14]:
pkldir

'/work/lotterhos/MVP-Offsets/1-trait_run_20220919_225-450/summaries/all_performance_dicts'

In [15]:
pkl = op.join(pkldir, 'level_scores.pkl')

pkldump(level_scores, pkl)

In [16]:
pkl

'/work/lotterhos/MVP-Offsets/1-trait_run_20220919_225-450/summaries/all_performance_dicts/level_scores.pkl'

In [17]:
formatclock(dt.now() - t0)

'0-00:00:02'